[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/release/docs/release/tutorials/generate-social-media-post-from-video-with-gradio-ui.ipynb) [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixeltable/pixeltable/blob/release/docs/release/tutorials/generate-social-media-post-from-video-with-gradio-ui.ipynb)

# Video Analysis and Social Media Post Creation with Pixeltable and Gradio

Also hosted on [Hugging Face](https://huggingface.co/spaces/Pixeltable/video-to-social-media-post-generator).

In this example, we'll demonstrate key Pixeltable functionalities for managing and processing video data, integrating AI models, and creating an interactive interface. We'll show how Pixeltable can be used to:

1. Create tables and views to store and organize video data;
2. Extract frames and audio from videos through automated processing;
3. Compute and store metadata, transcriptions, and AI-generated content;
4. Utilize OpenAI's GPT and Whisper models for transcription and content generation;
5. Define user-defined functions (UDFs) for specialized tasks like prompt construction;
6. Store transformed data and AI outputs for easy retrieval and analysis;
7. Create an interactive web interface using Gradio for easy user interaction with Pixeltable's functionalities.

This example assumes you're already somewhat familiar with Pixeltable.
If this is your first time using Pixeltable, the [Pixeltable Basics tutorial](https://docs.pixeltable.com/docs/pixeltable-basics) is a great place to start.

In [ ]:
%pip install -qU pixeltable gradio openai openai-whisper spacy

In [176]:
import pixeltable as pxt
import os
import openai
import gradio as gr
import getpass
from pixeltable.iterators import FrameIterator
from pixeltable.functions.video import extract_audio
from pixeltable.functions.audio import get_metadata
from pixeltable.functions import openai

## Store OpenAI API Key

In [177]:
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')

## Create a Table, a View, and Computed Columns

In [222]:
pxt.drop_dir('directory', force=True)
pxt.create_dir('directory')

t = pxt.create_table(
    'directory.video_table', {
    "video": pxt.VideoType(nullable=True),
    "sm_type": pxt.StringType(nullable=True),
    }
)

frames_view = pxt.create_view(
    "directory.frames",
    t,
    iterator=FrameIterator.create(video=t.video, fps=.25)
)

Created directory `directory`.
Created table `video_table`.
Created view `frames` with 0 rows, 0 exceptions.


In [223]:
# Create computed columns to store transformations and persist outputs
t['audio'] = extract_audio(t.video, format='mp3')
t['metadata'] = get_metadata(t.audio)
t['transcription'] = openai.transcriptions(audio=t.audio, model='whisper-1')
t['transcription_text'] = t.transcription.text

Added 0 column values with 0 errors.
Added 0 column values with 0 errors.
Added 0 column values with 0 errors.
Added 0 column values with 0 errors.


## Custom UDF for Generating Social Media Prompts

In [224]:
 #Custom User-Defined Function (UDF) for Generating Social Media Prompts
@pxt.udf
def prompt(A: str, B: str) -> list[dict]:
    system_msg = 'You are an expert in creating social media content and you generate effective post, based on user content. Respect the social media platform guidelines and constraints.'
    user_msg = f'A: "{A}" \n B: "{B}"'
    return [
        {'role': 'system', 'content': system_msg},
        {'role': 'user', 'content': user_msg}
    ]

# Apply the UDF to create a new column
t['message'] = prompt(t.sm_type, t.transcription_text)

Added 0 column values with 0 errors.


## Generating Responses with OpenAI's GPT Model

In [225]:
# # Generate responses using OpenAI's chat completion API
t['response'] = openai.chat_completions(messages=t.message, model='gpt-4o-mini-2024-07-18', max_tokens=500)

## Extract the content of the response
t['answer'] = t.response.choices[0].message.content

Added 0 column values with 0 errors.
Added 0 column values with 0 errors.


In [226]:
MAX_VIDEO_SIZE_MB = 35

def process_and_generate_post(video_file, social_media_type):
    if not video_file:
        return "Please upload a video file.", None

    try:
        # Check video file size
        video_size = os.path.getsize(video_file) / (1024 * 1024)  # Convert to MB
        if video_size > MAX_VIDEO_SIZE_MB:
            return f"The video file is larger than {MAX_VIDEO_SIZE_MB} MB. Please upload a smaller file.", None

        # # Insert a video into the table. Pixeltable supports referencing external data sources like URLs
        t.insert([{
            "video": video_file,
            "sm_type": social_media_type
        }])

        # Retrieve Social media posts
        social_media_post = t.select(t.answer).tail(1)['answer'][0]

        # Retrieve Audio
        audio = t.select(t.audio).tail(1)['audio'][0]

        # Retrieve thumbnails
        thumbnails = frames_view.select(frames_view.frame).tail(4)['frame']

        # Retrieve Pixeltable Table containing all videos and stored data
        df_output = t.collect().to_pandas()

        #Display content
        return social_media_post, thumbnails, df_output, audio

    except Exception as e:
        return f"An error occurred: {str(e)}", None

# Gradio Interface
import gradio as gr

def gradio_interface():
    with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
        gr.Markdown(
            """<p>
            <img src="https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/source/data/pixeltable-logo-large.png" alt="Pixeltable" width="20%" />
            <h1>Video to Social Media Post Generator</h1>
            <h3>Key functionalities demonstrated in this example:</h3>
            </p>
            <ul>
  <li><strong>Video Data Management:</strong> Creating tables and views to store and organize video data.</li>
  <li><strong>Automated Video Processing:</strong> Extracting frames and audio from videos.</li>
  <li><strong>Data Transformation:</strong> Computing and storing metadata, transcriptions, and AI-generated content.</li>
  <li><strong>AI Integration:</strong> Utilizing OpenAI's GPT and Whisper models for transcription and content generation.</li>
  <li><strong>Custom Functions:</strong> Defining user-defined functions (UDFs) for specialized tasks like prompt construction.</li>
  <li><strong>Data Persistence:</strong> Storing transformed data and AI outputs for easy retrieval and analysis.</li>
  <li><strong>Gradio Integration:</strong> Creating an interactive web interface for easy user interaction with Pixeltable's functionalities.</li>
  </ul>
            """
        )

        with gr.Row():
            with gr.Column():
                video_input = gr.Video(
                    label=f"Upload Video File (max {MAX_VIDEO_SIZE_MB} MB):",
                    include_audio=True,
                    max_length=300,
                    height='400px',
                    autoplay=False
                )
                social_media_type = gr.Dropdown(
                    choices=["X (Twitter)", "Facebook", "LinkedIn", "Instagram"],
                    label="Select Social Media Platform:",
                    value="X (Twitter)",
                )
                generate_btn = gr.Button("Generate Post")

                gr.Examples(
            examples=[["example1.mp4"], ["example2.mp4"], ["example3.mp4"]],
            inputs=[video_input]
        )
            with gr.Column():
                output = gr.Textbox(label="Generated Social Media Post", show_copy_button=True)
                thumbnail = gr.Gallery(
                    label="Pick your favorite Post Thumbnail",
                    show_download_button=True,
                    show_fullscreen_button=True,
                    height='400px'
                )
                audio = gr.Audio()

        df_output = gr.DataFrame(label="Pixeltable Table")

        generate_btn.click(
            fn=process_and_generate_post,
            inputs=[video_input, social_media_type],
            outputs=[output, thumbnail, df_output, audio],
        )

        gr.HTML(
            """
                <div class="footer">
                    <p>Pixeltable is a declarative interface for working with text, images, embeddings, and even video, enabling you to store, transform, index, and iterate on data. Powered solely by <a href="https://github.com/pixeltable/pixeltable" style="text-decoration: underline;" target="_blank">Pixeltable</a> - running OpenAI (gpt-4o-mini-2024-07-18).</a></p>
                <p><a href="https://colab.research.google.com/github/pixeltable/pixeltable/blob/release/docs/release/tutorials/pixeltable-basics.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a></p>
                </div>
           """
        )
    return demo

# Launch the Gradio interface
if __name__ == "__main__":
    gradio_interface().launch(show_api=False)


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://01b04b6f6c9665579d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



This example showcases how Pixeltable simplifies complex video processing workflows and integrates AI capabilities to create a powerful tool for generating social media content from video inputs.